This notebook will include an analysis of the Breast Classification Dataset, now consisting in a manually constructed Neural Networks with the help of Pytorch. 

## Importing Packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.datasets
from sklearn.model_selection import train_test_split

## Data Preprocessing

In [ ]:
bc_df = sklearn.datasets.load_breast_cancer() #loading the dataset; its in the form of a dictionary

In [ ]:
df = pd.DataFrame(bc_df.data, columns = bc_df.feature_names) #converting it to a proper dataframe

In [ ]:
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [ ]:
#adding the target variable to the df
df['label'] = bc_df.target

In [ ]:
df.info() #no missing values so far

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mean radius              569 non-null    float64
 1   mean texture             569 non-null    float64
 2   mean perimeter           569 non-null    float64
 3   mean area                569 non-null    float64
 4   mean smoothness          569 non-null    float64
 5   mean compactness         569 non-null    float64
 6   mean concavity           569 non-null    float64
 7   mean concave points      569 non-null    float64
 8   mean symmetry            569 non-null    float64
 9   mean fractal dimension   569 non-null    float64
 10  radius error             569 non-null    float64
 11  texture error            569 non-null    float64
 12  perimeter error          569 non-null    float64
 13  area error               569 non-null    float64
 14  smoothness error         5

## Split data into features and targets

In [ ]:
X = df.drop(columns = ['label'], axis = 1)
y = df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

##Standardize the data

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
standard_scaler = StandardScaler()

X_train_std = standard_scaler.fit_transform(X_train)
X_test_std = standard_scaler.fit_transform(X_test)
X_std = standard_scaler.fit_transform(X)

In [ ]:
#Convert dataframes into numpy arrays
y_train_array = y_train.to_numpy()
y_test_array = y_test.to_numpy()
y_array = y.to_numpy()

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [ ]:
input_size = X_train_std.shape[1]
hidden_size = 20
output_size = 2 #applying softmax 

We will create the class Data to convert the dataframes of our training and test sets into torch tensors. Also we will divide our sets into batches of 32. 

In [ ]:
#Convert the numpy arrays into tensors

y_train_list = list(y_train_array)

class MyDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X_std.astype(np.float32))
        self.y = torch.from_numpy(y_array.astype(np.float32))
        self.lengths = self.X.shape[0]

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

train_dataset = MyDataset(X_train_std, y_train_array)
test_dataset = MyDataset(X_test_std, y_test_array)


train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = True)

'''for batch, (X, y) in enumerate(loader):
    print("Batch: ",[batch+1])
    print("X shape: ", X.shape)
    print("y shape: ",y.shape)
    break'''

#number of batches
print(len(train_loader))

#batch size
for batch_inputs, batch_outputs in train_loader:
  print(len(batch_inputs))

9
64
64
64
64
64
64
64
64
57


## Neural Network Implementation

In [ ]:
class NeuralNetwork(nn.Module):
  
  def __init__(self, input_size, hidden_size, output_size):

    super(NeuralNetwork, self).__init__()

    #Initializing the units
    self.input_size = input_size #number of neurons in input layer
    self.hidden_size = hidden_size #number of neurons in the hidden layer
    self.output_size = output_size


    #Defining the layers
    self.layer_1 = nn.Linear(input_size, hidden_size)

    #Implementing relu function which will be applied to first layer
    self.relu = nn.ReLU()

    #initializing the weights of the neural network using the Kaiming uniform method
    nn.init.kaiming_uniform(self.layer_1.weight, nonlinearity = "relu")

    #Defining the output layer
    self.layer_2 = nn.Linear(hidden_size, output_size)

    #Implementing sigmoid function which will be applied to the output layer
    self.softmax = nn.Softmax()

  def forward(self, x):

    layer_1 = self.layer_1(x)
    relu = self.relu(layer_1)
    layer_2 = self.layer_2(relu)
    output = self.softmax(layer_2)
    
    return output

In [ ]:
model = NeuralNetwork(input_size, hidden_size, output_size)
print(model)

NeuralNetwork(
  (layer_1): Linear(in_features=30, out_features=20, bias=True)
  (relu): ReLU()
  (layer_2): Linear(in_features=20, out_features=2, bias=True)
  (softmax): Softmax(dim=None)
)


<ipython-input-16-8f844e9b8366>:20: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  nn.init.kaiming_uniform(self.layer_1.weight, nonlinearity = "relu")


## Optimization

In [ ]:
import torch.optim as optim

To calculate the model, we must define a loss function to calculate the gradients and an optimizer to update the parameters. We are going to use Binary Cross Entropy with Stochastic Gradient Descent and a learning rate of 0.001.

In [ ]:
#Initializing learning rate
learning_rate = 0.0001

#Initializing the Stochasting Gradient Descent, which will update the weights
stochastic = optim.Adam(model.parameters(), lr = learning_rate)


In [ ]:
#initialize the number of epochs
'''epochs = 10
loss_values = []

for epoch in range(epochs):

    #Set the gradients to zero
    stochastic.zero_grad()

    #Predict the model
    y_pred = model(X_train_tensor)
  
    #Get the loss
    y_train_tensor = y_train_tensor.long()

    loss = loss_function(y_pred, y_train_tensor)

    #Get the stats
    print(f"Epoch {epoch}: traing loss: {loss}")

    #Compute the gradients
    loss.backward()

    #Take a step to optimize the weights
    stochastic.step()

print(y_pred.shape)'''

'epochs = 10\nloss_values = []\n\nfor epoch in range(epochs):\n\n    #Set the gradients to zero\n    stochastic.zero_grad()\n\n    #Predict the model\n    y_pred = model(X_train_tensor)\n  \n    #Get the loss\n    y_train_tensor = y_train_tensor.long()\n\n    loss = loss_function(y_pred, y_train_tensor)\n\n    #Get the stats\n    print(f"Epoch {epoch}: traing loss: {loss}")\n\n    #Compute the gradients\n    loss.backward()\n\n    #Take a step to optimize the weights\n    stochastic.step()\n\nprint(y_pred.shape)'

In [ ]:

# Define a loss function, which computes to cross entropy loss
def loss_function(batch_outputs, batch_labels):   

    batch_lengths = np.array([9,64,64,64,64,64,64,57]) #size of batches
    # Calculate the loss for the whole batch
    cross_entropy = nn.CrossEntropyLoss()
    loss = cross_entropy(batch_outputs, batch_labels.long())

    # Rescale the loss
    loss = loss / np.sum(batch_lengths)

    return loss


In [ ]:
# Function that will be called in every epoch
def train_epoch(loss_function, optimizer, model, loader):
  
  # Keep track of the total loss for the batch
  total_loss = 0
  total_loss = 0
  correct_predictions = 0
  total_samples = 0


  for X, y in train_loader:
    # Clear the gradients
    optimizer.zero_grad()
    # Run a forward pass
    pred = model.forward(X)
    # Compute the batch loss
    loss = loss_function(pred, y)
    # Calculate the gradients
    loss.backward()
    # Update the parameteres
    optimizer.step()
    total_loss += loss.item()

    correct_predictions += (pred.argmax(dim=1) == y).sum().item()
    total_samples += y.size(0)

    accuracy = (correct_predictions / total_samples) * 100


  return total_loss, accuracy


# Function containing our main training loop
def train(loss_function, optimizer, model, loader, num_epochs=10000):

  # Iterate through each epoch and call our train_epoch function
  for epoch in range(num_epochs):
    epoch_loss, accuracy = train_epoch(loss_function, optimizer, model, loader)
    #if epoch % 100 == 0: print(epoch_loss)
    print("Epoch: ",epoch," training loss: ",epoch_loss, "Accuracy: ", accuracy)

In [ ]:
#Evaluating loss

In [ ]:
num_epochs = 15
train(loss_function, stochastic, model, train_loader, num_epochs=num_epochs)

<ipython-input-16-8f844e9b8366>:33: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = self.softmax(layer_2)


Epoch:  0  training loss:  0.01119312981609255 Accuracy:  90.15817223198594
Epoch:  1  training loss:  0.011118452181108296 Accuracy:  90.50966608084359
Epoch:  2  training loss:  0.011041403515264392 Accuracy:  90.68541300527241
Epoch:  3  training loss:  0.01096855802461505 Accuracy:  91.03690685413005
Epoch:  4  training loss:  0.010895174113102257 Accuracy:  91.56414762741653
Epoch:  5  training loss:  0.010820519528351724 Accuracy:  91.91564147627417
Epoch:  6  training loss:  0.01075369876343757 Accuracy:  92.09138840070298
Epoch:  7  training loss:  0.01067732262890786 Accuracy:  92.2671353251318
Epoch:  8  training loss:  0.010608390788547695 Accuracy:  92.44288224956063
Epoch:  9  training loss:  0.010545849218033254 Accuracy:  92.61862917398945
Epoch:  10  training loss:  0.010479917051270604 Accuracy:  92.79437609841827
Epoch:  11  training loss:  0.010414323769509792 Accuracy:  93.32161687170475
Epoch:  12  training loss:  0.010348358773626387 Accuracy:  93.67311072056239
E